In [ ]:
# | default_exp _helpers.api_docs_helper

In [ ]:
# | export

from typing import *
import types
import enum
import ast
import textwrap
from pathlib import Path
from ast import ClassDef, FunctionDef, alias, ImportFrom
from inspect import signature, isfunction, isclass, getmembers, getdoc, getsource, getsourcefile
from inspect import Parameter as Parameter_Inspect
from importlib import import_module

import griffe
from griffe.dataclasses import Docstring, Function, Parameters, Parameter, ParameterKind, Module, Class
from griffe.docstrings.parsers import Parser, parse
from griffe.expressions import Name
from griffe.agents.nodes import get_annotation, relative_to_absolute
from mkdocstrings_handlers.python.handler import get_handler, PythonHandler
from markdown.core import Markdown
from nbdev.config import get_config

In [ ]:
from subprocess import CalledProcessError
import unittest.mock
from contextlib import contextmanager

from nbdev.doclinks import NbdevLookup

from nbdev_mkdocs.mkdocs import prepare

In [ ]:
# | export


def _get_sample_markdown_handler_config() -> Markdown:
    md_config = Markdown(
        extensions=["toc"],
        extension_configs={},
    )
    return md_config

In [ ]:
sample_md_config = _get_sample_markdown_handler_config()

print(sample_md_config)
assert isinstance(sample_md_config, Markdown)

<markdown.core.Markdown object>


In [ ]:
# | export


def _get_handler(md_config: Markdown) -> PythonHandler:
    handler = get_handler(theme="material", )
    handler._update_env(md_config, {'mdx': [], 'mdx_configs': []}) # check what config i need to pass
    return handler

In [ ]:
handler = _get_handler(sample_md_config)

print(handler.env.filters["convert_markdown"])
assert handler.env.filters["convert_markdown"]

<bound method BaseRenderer.do_convert_markdown of <mkdocstrings_handlers.python.handler.PythonHandler object>>


In [ ]:
# | export

class ParameterKindMapper(enum.Enum):
    POSITIONAL_ONLY: ParameterKind = ParameterKind.positional_only
    POSITIONAL_OR_KEYWORD: ParameterKind = ParameterKind.positional_or_keyword
    VAR_POSITIONAL: ParameterKind = ParameterKind.var_positional
    KEYWORD_ONLY: ParameterKind = ParameterKind.keyword_only
    VAR_KEYWORD: ParameterKind = ParameterKind.var_keyword

In [ ]:
assert ParameterKindMapper["POSITIONAL_OR_KEYWORD"].value == ParameterKind.positional_or_keyword
assert ParameterKindMapper["KEYWORD_ONLY"].value == ParameterKind.keyword_only

In [ ]:
# | export


def _get_symbol_filepath(symbol: Union[types.FunctionType, Type[Any]]) -> Path:
    config = get_config()
    filepath = getsourcefile(symbol)
    return Path(filepath).relative_to( # type: ignore
        filepath.split(f'{config["lib_name"].replace("-", "_")}/')[0] # type: ignore
    )

In [ ]:
actual = _get_symbol_filepath(prepare)
expected = Path("nbdev_mkdocs/mkdocs.py")
print(actual)

assert actual == expected

nbdev_mkdocs/mkdocs.py


In [ ]:
# | export


def _get_module_source(module_name: str) -> str:
    m = import_module(module_name)
    return getsource(m)


def _get_absolute_module_import_path(
    symbol: Union[types.FunctionType, Type[Any]]
) -> Dict[str, str]:
    m_source = _get_module_source(symbol.__module__)
    tree = ast.parse(m_source)
    filepath = _get_symbol_filepath(symbol)
    name = ".".join(symbol.__module__.split(".")[:-1])
    current_module = Module(name=name, filepath=Path(filepath))
    return {
        name.name: relative_to_absolute(node, name, current_module)
        for node in tree.body
        if isinstance(node, ImportFrom)
        for name in node.names
    }

In [ ]:
actual = _get_absolute_module_import_path(prepare)
print(actual)

{'ConfigParser': 'configparser.ConfigParser', 'getmembers': 'inspect.getmembers', 'getmodule': 'inspect.getmodule', 'isclass': 'inspect.isclass', 'iscoroutine': 'inspect.iscoroutine', 'isfunction': 'inspect.isfunction', 'ismethod': 'inspect.ismethod', 'Path': 'pathlib.Path', '*': 'typing.*', 'ConfigUpdater': 'configupdater.ConfigUpdater', 'Section': 'configupdater.Section', 'Option': 'configupdater.option.Option', 'merge': 'fastcore.basics.merge', 'L': 'fastcore.foundation.L', 'move': 'fastcore.shutil.move', 'nbdev_clean': 'nbdev.clean.nbdev_clean', 'NbdevLookup': 'nbdev.doclinks.NbdevLookup', 'nbdev_export': 'nbdev.doclinks.nbdev_export', 'FrontmatterProc': 'nbdev.frontmatter.FrontmatterProc', '_fm2dict': 'nbdev.frontmatter._fm2dict', 'NBProcessor': 'nbdev.process.NBProcessor', 'prepare': 'nbdev.quarto.prepare', 'refresh_quarto_yml': 'nbdev.quarto.refresh_quarto_yml', 'proc_nbs': 'nbdev.serve.proc_nbs', 'nbdev_test': 'nbdev.test.nbdev_test', 'generate_cli_doc': 'nbdev_mkdocs._helpers.

In [ ]:
# | export


def _create_name_object(
    annotated_args: Dict[str, Any], abs_import_path: Dict[str, str]
) -> Dict[str, Any]:
    return {
        key: (
            Name(source=annotated_args[key].source, full=abs_import_path[val.full])
            if (isinstance(val, Name)) and (val.full in abs_import_path.keys())
            else val
        )
        for key, val in annotated_args.items()
    }

In [ ]:
annotated_args = {
    "arg_1": Name(source="str", full="str"),
    "arg_4": Name(source="CalledProcessError", full="CalledProcessError"),
    "arg_5": Name(source="prepare", full="prepare"),
    "arg_6": Name(source="NbdevLookup", full="NbdevLookup"),
    "arg_7": [
        Name(source="Union", full="Union"),
        "[",
        [
            [
                Name(source="List", full="List"),
                "[",
                Name(source="str", full="str"),
                "]",
            ],
            ", ",
            Name(source="str", full="str"),
        ],
        "]",
    ],
    "kwargs": None,
}
abs_import_path = {
    "CalledProcessError": "subprocess.CalledProcessError",
    "NbdevLookup": "nbdev.doclinks.NbdevLookup",
    "set_cwd": "nbdev_mkdocs._helpers.utils.set_cwd",
    "prepare": "nbdev_mkdocs.mkdocs.prepare",
}

actual = _create_name_object(annotated_args, abs_import_path)
expected = {
    "arg_1": Name(source="str", full="str"),
    "arg_4": Name(source="CalledProcessError", full="subprocess.CalledProcessError"),
    "arg_5": Name(source="prepare", full="nbdev_mkdocs.mkdocs.prepare"),
    "arg_6": Name(source="NbdevLookup", full="nbdev.doclinks.NbdevLookup"),
    "arg_7": [
        Name(source="Union", full="Union"),
        "[",
        [
            [
                Name(source="List", full="List"),
                "[",
                Name(source="str", full="str"),
                "]",
            ],
            ", ",
            Name(source="str", full="str"),
        ],
        "]",
    ],
    "kwargs": None,
}
print(actual)
assert actual == expected

{'arg_1': Name(source='str', full='str'), 'arg_4': Name(source='CalledProcessError', full='subprocess.CalledProcessError'), 'arg_5': Name(source='prepare', full='nbdev_mkdocs.mkdocs.prepare'), 'arg_6': Name(source='NbdevLookup', full='nbdev.doclinks.NbdevLookup'), 'arg_7': [Name(source='Union', full='Union'), '[', [[Name(source='List', full='List'), '[', Name(source='str', full='str'), ']'], ', ', Name(source='str', full='str')], ']'], 'kwargs': None}


In [ ]:
# | export


def _flattern(xs: List[Any]) -> List[Any]:
    return [subitem for item in xs if isinstance(item, list) for subitem in item] + [
        item for item in xs if not isinstance(item, list)
    ]


def _get_init_node(node: ClassDef) -> Optional[FunctionDef]:
    init_node = [
        n for n in node.body if isinstance(n, FunctionDef) and n.name == "__init__"
    ]
    if len(init_node) == 0:
        return None
    return init_node[0]


def _get_annotation_for_all_params(symbol: Union[types.FunctionType, Type[Any]]) -> Dict[str, Any]:
    source = getsource(symbol)
    tree = ast.parse(textwrap.dedent(source))
    name = ".".join(symbol.__module__.split(".")[:-1])
    parent = Module(
        name=name,
    )
    node = (
        _get_init_node(tree.body[0])
        if isinstance(tree.body[0], ClassDef)
        else tree.body[0]
    )

    if node is None:
        return {}

    args_list = {
        key: value
        for key, value in node.args.__dict__.items() # type: ignore
        if key not in ["defaults", "kw_defaults"]
    }
    args = [arg for arg in args_list.values() if arg and arg != [None]]
    annotated_args = {
        arg.arg: get_annotation(arg.annotation, parent) for arg in _flattern(args)
    }
    abs_import_path = _get_absolute_module_import_path(symbol)
    return _create_name_object(annotated_args, abs_import_path)

In [ ]:
@contextmanager
def mock_get_module_source(get_module_source_mock_value):
    with unittest.mock.patch('__main__._get_module_source') as mock_get_module_source:
        mock_get_module_source.return_value = get_module_source_mock_value
        yield

In [ ]:
def fixture_function(
    arg_1: str,
    arg_2: int,
    arg_3: bool,
    arg_4: CalledProcessError,
    arg_5: prepare,
    arg_6: NbdevLookup,
    arg_7: Union[List[str], str],
    arg_8: Optional[int],
    arg_9: Optional[str] = None,
    **kwargs
) -> str:
    """This is a one line description for the function

    Args:
        arg_1: Argument 1
        arg_2: Argument 2
        arg_3: Argument 3
        arg_4: Argument 4
        arg_5: Argument 5
        arg_6: Argument 6
        arg_7: Argument 5
        arg_8: Argument 6
        arg_9: Argument 5
        **kwargs: arguments use to replace "{fill in **param**}" in docstring with the actual values when running examples

    Returns:
        The concatinated string
    """
    pass

expected = {'arg_1': Name(source='str', full='str'), 'arg_2': Name(source='int', full='int'), 'arg_3': Name(source='bool', full='bool'), 'arg_4': Name(source='CalledProcessError', full='subprocess.CalledProcessError'), 'arg_5': Name(source='prepare', full='nbdev_mkdocs.mkdocs.prepare'), 'arg_6': Name(source='NbdevLookup', full='nbdev.doclinks.NbdevLookup'), 'arg_7': [Name(source='Union', full='Union'), '[', [[Name(source='List', full='List'), '[', Name(source='str', full='str'), ']'], ', ', Name(source='str', full='str')], ']'], 'arg_8': [Name(source='Optional', full='Optional'), '[', Name(source='int', full='int'), ']'], 'arg_9': [Name(source='Optional', full='Optional'), '[', Name(source='str', full='str'), ']'], 'kwargs': None}


get_module_source_value = """
from subprocess import CalledProcessError, run
from nbdev_mkdocs.mkdocs import prepare
from nbdev.doclinks import NbdevLookup

def fixture_function(
    arg_1: str,
    arg_2: int,
    arg_3: bool,
    arg_4: CalledProcessError,
    arg_5: prepare,
    arg_6: NbdevLookup,
    arg_7: Union[List[str], str],
    arg_8: Optional[int],
    arg_9: Optional[str] = None,
    **kwargs
) -> str:
    \"\"\"This is a one line description for the function

    Args:
        arg_1: Argument 1
        arg_2: Argument 2
        arg_3: Argument 3
        arg_4: Argument 4
        arg_5: Argument 5
        arg_6: Argument 6
        arg_7: Argument 5
        arg_8: Argument 6
        arg_9: Argument 5
        **kwargs: arguments use to replace "{fill in **param**}" in docstring with the actual values when running examples

    Returns:
        The concatinated string
    \"\"\"
    pass
"""

with mock_get_module_source(get_module_source_value):
    actual = _get_annotation_for_all_params(fixture_function)

print(actual)
assert actual == expected

{'arg_1': Name(source='str', full='str'), 'arg_2': Name(source='int', full='int'), 'arg_3': Name(source='bool', full='bool'), 'arg_4': Name(source='CalledProcessError', full='subprocess.CalledProcessError'), 'arg_5': Name(source='prepare', full='nbdev_mkdocs.mkdocs.prepare'), 'arg_6': Name(source='NbdevLookup', full='nbdev.doclinks.NbdevLookup'), 'arg_7': [Name(source='Union', full='Union'), '[', [[Name(source='List', full='List'), '[', Name(source='str', full='str'), ']'], ', ', Name(source='str', full='str')], ']'], 'arg_8': [Name(source='Optional', full='Optional'), '[', Name(source='int', full='int'), ']'], 'arg_9': [Name(source='Optional', full='Optional'), '[', Name(source='str', full='str'), ']'], 'kwargs': None}


In [ ]:
@contextmanager
def mock_getsource(mock_value):
    with unittest.mock.patch('__main__.getsource') as mock_getsource:
        mock_getsource.return_value = mock_value
        yield

In [ ]:
getsourcefile_mock_value = '/Users/harishm/Dev/airt-git/nbdev-mkdocs/nbdev_mkdocs/mkdocs.py'

@contextmanager
def mock_getsourcefile():
    with unittest.mock.patch('__main__.getsourcefile') as mock_getsourcefile:
        mock_getsourcefile.return_value = getsourcefile_mock_value
        yield

In [ ]:
class Car:
    pass

getsource_mock_value = """class Fixture:
    \"\"\"Some docstring\"\"\"
    def __init__(self, make: str, model: str, year: int, color: str):
        \"\"\"Constructor\"\"\"
        pass
    def method_one(self, a: str) -> str:
        \"\"\"Method one
        
        Args:
            a: A string
        
        Returns:
            A string
        \"\"\"
        return "hello world"
"""

expected = {'self': None, 'make': Name(source='str', full='str'), 'model': Name(source='str', full='str'), 'year': Name(source='int', full='int'), 'color': Name(source='str', full='str')}

with mock_getsourcefile():
    with mock_getsource(getsource_mock_value):
        actual = _get_annotation_for_all_params(Car)
print(actual)
assert actual == expected

{'self': None, 'make': Name(source='str', full='str'), 'model': Name(source='str', full='str'), 'year': Name(source='int', full='int'), 'color': Name(source='str', full='str')}


In [ ]:
class Car:
    pass

getsource_mock_value = """class Fixture:
    \"\"\"Some docstring\"\"\"
    def method_one(self, a: str) -> str:
        \"\"\"Method one
        
        Args:
            a: A string
        
        Returns:
            A string
        \"\"\"
        return "hello world"
"""
expected = {}
with mock_getsource(getsource_mock_value):
    actual = _get_annotation_for_all_params(Car)
print(actual)
assert actual == expected

{}


In [ ]:
class Car:
    pass

getsource_mock_value = """class Fixture:
    def __init__(self):
        \"\"\"Constructor\"\"\"
        pass
    def method_one(self, a: str) -> str:
        \"\"\"Method one
        
        Args:
            a: A string
        
        Returns:
            A string
        \"\"\"
        return "hello world"
"""
with mock_getsourcefile():
    with mock_getsource(getsource_mock_value):
        actual = _get_annotation_for_all_params(Car)
print(actual)

{'self': None}


In [ ]:
def fixture_function(param1: str, *, param2: int, **kwargs) -> str:
    """Hello I'm a docstring!

    Args:
        param1: Description.
        param2: Description.
        
    Returns:
        A string
    """
    pass

expected = {'param1': Name(source='str', full='str'), 'param2': Name(source='int', full='int'), 'kwargs': None}

get_module_source_value = """
def fixture_function(param1: str, *, param2: int, **kwargs) -> str:
    \"\"\"Hello I'm a docstring!

    Args:
        param1: Description.
        param2: Description.
        
    Returns:
        A string
    \"\"\"
    pass
"""
with mock_get_module_source(get_module_source_value):
    actual = _get_annotation_for_all_params(fixture_function)

print(actual)
assert actual == expected

{'param1': Name(source='str', full='str'), 'param2': Name(source='int', full='int'), 'kwargs': None}


In [ ]:
# | export


def _get_return_annotation(symbol: Union[types.FunctionType, Type[Any]]) -> Optional[Name]:
    sig = signature(symbol)
    if sig.return_annotation is None or sig.return_annotation.__name__ == "_empty":
        return None

    return_signature = sig.return_annotation.__name__
    abs_import_path = _get_absolute_module_import_path(symbol)
    full_return_signature = (
        abs_import_path[return_signature]
        if return_signature in abs_import_path.keys()
        else return_signature
    )
    return Name(source=return_signature, full=full_return_signature)

In [ ]:
def fixture_function(param1: str) -> prepare:
    """Hello I'm a docstring!

    Args:
        param1: Description.
        param2: Description.
        
    Returns:
        A string
    """
    pass

get_module_source_value = """
from subprocess import CalledProcessError, run
from nbdev_mkdocs.mkdocs import prepare
from nbdev.doclinks import NbdevLookup

def fixture_function(
    param1: str,
) -> prepare:
    \"\"\"This is a one line description for the function

    Args:
        param1: Argument 1

    Returns:
        prepare instance
    \"\"\"
    pass
"""

expected = Name(source="prepare", full="nbdev_mkdocs.mkdocs.prepare")

with mock_get_module_source(get_module_source_value):
    actual = _get_return_annotation(fixture_function)

print(actual)
print(f"actual.source = {actual.source}")
print(f"actual.full = {actual.full}")
assert actual == expected

prepare
actual.source = prepare
actual.full = nbdev_mkdocs.mkdocs.prepare


In [ ]:
def fixture_function(param1: str, *, param2: int, **kwargs) -> None:
    """Hello I'm a docstring!

    Args:
        param1: Description.
        param2: Description.
        
    Returns:
        A string
    """
    pass

actual = _get_return_annotation(fixture_function)
expected = None
print(actual)
assert actual == expected

None


In [ ]:
# | export


def _get_param_default_value(param: Parameter_Inspect) -> Optional[str]:
    return (
        "{}"
        if param.name == "kwargs"
        else None
        if param.default is param.empty
        else str(param.default)
    )


def _get_parameters(
    symbol: Union[types.FunctionType, Type[Any]]
) -> Optional[Parameters]:
    sig = signature(symbol)
    params = [param for param in sig.parameters.values()]
    annotations = _get_annotation_for_all_params(symbol)
    parameters = [
        Parameter(
            param.name,
            annotation=annotations[param.name],
            default=_get_param_default_value(param),
            kind=ParameterKindMapper[param.kind.name].value,
        )
        for param in params
    ]
    if len(parameters) == 0:
        return None
    return Parameters(*parameters)

In [ ]:
def fixture_function(param1: str, *, param2: int, **kwargs) -> str:
    """Hello I'm a docstring!

    Args:
        param1: Description.
        param2: Description.
        
    Returns:
        A string
    """
    pass

expected = Parameters(
    Parameter("param1", annotation=Name(source="str",full="str"), kind=ParameterKind.positional_or_keyword),
    Parameter("param2", annotation=Name(source="int",full="int"), kind=ParameterKind.keyword_only),
    Parameter("kwargs", kind=ParameterKind.var_keyword, default="{}"),
)

get_module_source_value = """
def fixture_function(param1: str, *, param2: int, **kwargs) -> str:
    \"\"\"Hello I'm a docstring!

    Args:
        param1: Description.
        param2: Description.
        
    Returns:
        A string
    \"\"\"
    pass
"""

with mock_get_module_source(get_module_source_value):
    actual = _get_parameters(fixture_function)

print(actual)

sig = signature(fixture_function)
for param in sig.parameters:
    print(param)
    assert actual._parameters_dict[param].as_dict() == expected._parameters_dict[param].as_dict()

<griffe.dataclasses.Parameters object>
param1
param2
kwargs


In [ ]:
class Car:
    pass

getsource_mock_value = """class Fixture:
    \"\"\"Some docstring\"\"\"
    def method_one(self, a: str) -> str:
        \"\"\"Method one
        
        Args:
            a: A string
        
        Returns:
            A string
        \"\"\"
        return "hello world"
"""
with mock_getsource(getsource_mock_value):
    actual = _get_parameters(Car)
print(actual)
assert actual == None

None


In [ ]:
# | export


def _get_object_for_symbol(
    symbol: Union[types.FunctionType, Type[Any]]
) -> Union[Class, Function]:
    if isfunction(symbol):
        return Function(
            symbol.__name__,
            parameters=_get_parameters(symbol),
            returns=_get_return_annotation(symbol),
        )
    return Class(symbol.__name__)

In [ ]:
def fixture_function(param1: str, *, param2: int, **kwargs) -> str:
    """Hello I'm a docstring!

    Args:
        param1: Description.
        param2: Description.
        
    Returns:
        A string
    """
    pass

get_module_source_value = """
def fixture_function(param1: str, *, param2: int, **kwargs) -> str:
    \"\"\"Hello I'm a docstring!

    Args:
        param1: Description.
        param2: Description.
        
    Returns:
        A string
    \"\"\"
    pass
"""

with mock_get_module_source(get_module_source_value):
    actual = _get_object_for_symbol(fixture_function)

print(actual)
assert isinstance(actual, Function)

<Function('fixture_function', None, None)>


In [ ]:
class T:
    pass

actual = _get_object_for_symbol(T)

print(actual)
assert isinstance(actual, Class)

<Class('T', None, None)>


In [ ]:
# | export

def _add_symbol_source_to_docstring(symbol: Union[types.FunctionType, Type[Any]]) -> str:
    source_code = getsource(symbol)
    symbol_path = _get_symbol_filepath(symbol)
    formatted_sourcecode = textwrap.dedent(source_code)
    return (
            f'??? quote "Source code in `{symbol_path}`"\n\n'
            + f"    ```python\n{textwrap.indent(formatted_sourcecode, '    ')}    ```\n\n"
        )

In [ ]:
actual = _add_symbol_source_to_docstring(prepare)

print(actual)
assert actual

??? quote "Source code in `nbdev_mkdocs/mkdocs.py`"

    ```python
    def prepare(
        root_path: str,
        use_relative_doc_links: bool = False,
        no_test: bool = False,
        no_mkdocs_build: bool = False,
    ) -> None:
        """Prepare mkdocs for serving

        Args:
            root_path: The root path of the project
            use_relative_doc_links: If set to True, relative links are added to symbol references in generated
                documentation. Else, the value set in doc_host in settings.ini is added to symbol references in
                generated documentation. This flag should be set to `False` if this function is called directly
                without calling preview.
            no_test: If set to False, the unit tests will be run, else they will be skipped
            no_mkdocs_build: If set to True, then the mkdocs build will be skipped. This flag
                should be set to `False` if this function is called directly without calling p

In [ ]:
# | export


def _generate_markup_for_docstring_section(section: Any, handler: PythonHandler) -> str:
    template = handler.env.get_template(f"docstring/{section.kind.value}.html")
    rendered_html = template.render(section=section, config=handler.default_config)
    return f"{rendered_html}\n"


def _docstring_to_markdown(symbol: Union[types.FunctionType, Type[Any]]) -> str:
    """Converts a docstring to a markdown-formatted string.

    Args:
        symbol: The symbol for which the docstring needs to be converted.

    Returns:
        The markdown-formatted docstring.
    """
    parent = _get_object_for_symbol(symbol)
    docstring = Docstring(getdoc(symbol), parent=parent)  # type: ignore
    parsed_docstring_sections = parse(docstring, Parser.google)

    md_config = _get_sample_markdown_handler_config()
    handler = _get_handler(md_config)

    parsed_sections = [
        f"{section.value}\n" # type: ignore
        if section.kind.value == "text"
        else _generate_markup_for_docstring_section(section, handler)
        for section in parsed_docstring_sections
    ]
    
    ret_val = "".join(parsed_sections) + f"\n\n{_add_symbol_source_to_docstring(symbol)}"
    
    return ret_val

In [ ]:
get_module_source_value="""
def fixture_function(i: str, a: str) -> str:
    \"\"\"Very cool function

    **f** is a very cool function
    
    Note:
        Execution context is not the same as the one in the notebook because we want examples to work from
        user code. Make sure you compiled the library prior to executing the examples, otherwise you might
        be running them agains an old version of the library.

    Args:
        i: something
        a: something else
        
    Returns:
        This function returns a sample string

    Raises:
        ValueError: If the object has no docstring
        HTTPError: If the object has no docstring
        
    Examples:
        The following snippet prints out greetings for two names:
        ```python
        print("hello {fill in name_1}")
        print("goodbye {fill in name_2}")
        ```
        
    Example:
        The following snippet prints out greetings for two names:
        ```python
        print("hello {fill in name_1}")
        print("goodbye {fill in name_2}")
        ```
        
    Yields:
        This functiron yields something

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)

    \"\"\"
    pass
"""

with mock_get_module_source(get_module_source_value):
    actual = _docstring_to_markdown(fixture_function)
print(actual)

griffe: <module>:5: Failed to get 'name: description' pair from ''


Hello I'm a docstring!

  <p><strong>Parameters:</strong></p>
  <table>
    <thead>
      <tr>
        <th>Name</th>
        <th>Type</th>
        <th>Description</th>
        <th>Default</th>
      </tr>
    </thead>
    <tbody>
        <tr>
          <td><code>param1</code></td>
          <td>
                <code><span data-autorefs-optional="str">str</span></code>
          </td>
          <td><p>Description.</p></td>
          <td>
              <em>required</em>
          </td>
        </tr>
        <tr>
          <td><code>param2</code></td>
          <td>
                <code><span data-autorefs-optional="int">int</span></code>
          </td>
          <td><p>Description.</p></td>
          <td>
              <em>required</em>
          </td>
        </tr>
    </tbody>
  </table>


  <p><strong>Returns:</strong></p>
  <table>
    <thead>
      <tr>
        <th>Type</th>
        <th>Description</th>
      </tr>
    </thead>
    <tbody>
        <tr>
          <td>
            

In [ ]:
# | export

MKDOCS_LOCAL_CONFIG = f"""
    options:
      members: false
      show_docstring_attributes: false
      show_docstring_description: false
      show_docstring_examples: false
      show_docstring_other_parameters: false
      show_docstring_parameters: false
      show_docstring_raises: false
      show_docstring_receives: false
      show_docstring_returns: false
      show_docstring_warns: false
      show_docstring_yields: false
      show_source: false\n\n"""


def _get_annotated_symbol_definition(symbol: Union[types.FunctionType, Type[Any]]) -> str:
    try:
        module = f"{symbol.__module__}.{symbol.__qualname__}"
        griffe.load(module)
        return f"\n\n::: {module}{MKDOCS_LOCAL_CONFIG}"
    except KeyError as e:
        patched_symbol_path = _get_symbol_filepath(symbol)
        return f"\n\n::: {str(patched_symbol_path.parent)}.{str(patched_symbol_path.stem)}.{symbol.__name__}{MKDOCS_LOCAL_CONFIG}"

In [ ]:
actual = _get_annotated_symbol_definition(prepare)
expected = """

::: nbdev_mkdocs.mkdocs.prepare
    options:
      members: false
      show_docstring_attributes: false
      show_docstring_description: false
      show_docstring_examples: false
      show_docstring_other_parameters: false
      show_docstring_parameters: false
      show_docstring_raises: false
      show_docstring_receives: false
      show_docstring_returns: false
      show_docstring_warns: false
      show_docstring_yields: false
      show_source: false

"""
print(actual)
assert actual == expected



::: nbdev_mkdocs.mkdocs.prepare
    options:
      members: false
      show_docstring_attributes: false
      show_docstring_description: false
      show_docstring_examples: false
      show_docstring_other_parameters: false
      show_docstring_parameters: false
      show_docstring_raises: false
      show_docstring_receives: false
      show_docstring_returns: false
      show_docstring_warns: false
      show_docstring_yields: false
      show_source: false




In [ ]:
# | export



def get_formatted_docstring_for_symbol(
    symbol: Union[types.FunctionType, Type[Any]]
) -> str:
    """Recursively parses and get formatted docstring of a symbol.

    Args:
        symbol: A Python class or function object to parse the docstring for.

    Returns:
        A formatted docstring of the symbol and its members.

    """

    def traverse(symbol: Union[types.FunctionType, Type[Any]], contents: str) -> str:
        """Recursively traverse the members of a symbol and append their docstrings to the provided contents string.

        Args:
            symbol: A Python class or function object to parse the docstring for.
            contents: The current formatted docstrings.

        Returns:
            The updated formatted docstrings.

        """
        for x, y in getmembers(symbol):
            if not x.startswith("_") or x == "__init__":
                if isfunction(y) and y.__doc__ is not None:
                    contents += f"\n\n{_get_annotated_symbol_definition(y)}\n\n{_docstring_to_markdown(y)}"
                elif isclass(y) and not x.startswith("__") and y.__doc__ is not None:
                    contents += f"{_docstring_to_markdown(y)}"
                    contents = traverse(y, contents)
        return contents

    contents = (
        f"{_docstring_to_markdown(symbol)}"
        if symbol.__doc__ is not None
        else ""
    )
    if isclass(symbol):
        contents = traverse(symbol, contents)

    contents = f"::: {symbol.__module__}.{symbol.__qualname__}{MKDOCS_LOCAL_CONFIG}" + contents
    return contents

In [ ]:
_get_annotated_symbol_definition_mock_value = f'::: nbdev_mkdocs.mkdocs.prepare{MKDOCS_LOCAL_CONFIG}'

@contextmanager
def mock_get_annotated_symbol_definition():
    with unittest.mock.patch('__main__._get_annotated_symbol_definition') as mock_get_annotated_symbol_definition:
        mock_get_annotated_symbol_definition.return_value = _get_annotated_symbol_definition_mock_value
        yield


In [ ]:
getsource_mock_value = """class Car:
    \"\"\"A class representing a car.

    Attributes:
        make: The make of the car.
        model: The model of the car.
        year: The year the car was made.
        color: The color of the car.
    \"\"\"

    def __init__(self, make: str, model: str, year: int, color: str):
        \"\"\"Initialize a new car.

        Args:
            make: The make of the car.
            model: The model of the car.
            year: The year the car was made.
            color: The color of the car.
        \"\"\"
        self.make = make
        self.model = model
        self.year = year
        self.color = color
        self.is_running = False
"""

class Car:
    """A class representing a car.

    Attributes:
        make: The make of the car.
        model: The model of the car.
        year: The year the car was made.
        color: The color of the car.
    """

    def __init__(self, make: str, model: str, year: int, color: str):
        """Initialize a new car.

        Args:
            make: The make of the car.
            model: The model of the car.
            year: The year the car was made.
            color: The color of the car.
        """
        self.make = make
        self.model = model
        self.year = year
        self.color = color
        self.is_running = False

with mock_getsourcefile():
    with mock_getsource(getsource_mock_value):
        with mock_get_annotated_symbol_definition():
            actual = get_formatted_docstring_for_symbol(Car)

print(actual)

::: __main__.Car
    options:
      members: false
      show_docstring_attributes: false
      show_docstring_description: false
      show_docstring_examples: false
      show_docstring_other_parameters: false
      show_docstring_parameters: false
      show_docstring_raises: false
      show_docstring_receives: false
      show_docstring_returns: false
      show_docstring_warns: false
      show_docstring_yields: false
      show_source: false

A class representing a car.

  <p><strong>Attributes:</strong></p>
  <table>
    <thead>
      <tr>
        <th>Name</th>
        <th>Type</th>
        <th>Description</th>
      </tr>
    </thead>
    <tbody>
        <tr>
          <td><code>make</code></td>
          <td>
          </td>
          <td><p>The make of the car.</p></td>
        </tr>
        <tr>
          <td><code>model</code></td>
          <td>
          </td>
          <td><p>The model of the car.</p></td>
        </tr>
        <tr>
          <td><code>year</code></td>


In [ ]:
get_module_source_value = """
def fixture_function(
    arg_1: str,
    arg_2: Union[List[str], str],
    arg_3: Optional[int],
    arg_4: Optional[str] = None,
) -> str:
    \"\"\"This is a one line description for the function

    Args:
        arg_1: Argument 1
        arg_2: Argument 2
        arg_3: Argument 3
        arg_4: Argument 4

    Returns:
        The concatinated string
    \"\"\"
    pass
"""

with mock_get_module_source(get_module_source_value):
    with mock_get_annotated_symbol_definition():
        actual = get_formatted_docstring_for_symbol(fixture_function)
print(actual)


griffe: <module>:5: Failed to get 'name: description' pair from ''


::: __main__.fixture_function
    options:
      members: false
      show_docstring_attributes: false
      show_docstring_description: false
      show_docstring_examples: false
      show_docstring_other_parameters: false
      show_docstring_parameters: false
      show_docstring_raises: false
      show_docstring_receives: false
      show_docstring_returns: false
      show_docstring_warns: false
      show_docstring_yields: false
      show_source: false

Hello I'm a docstring!

  <p><strong>Parameters:</strong></p>
  <table>
    <thead>
      <tr>
        <th>Name</th>
        <th>Type</th>
        <th>Description</th>
        <th>Default</th>
      </tr>
    </thead>
    <tbody>
        <tr>
          <td><code>param1</code></td>
          <td>
                <code><span data-autorefs-optional="str">str</span></code>
          </td>
          <td><p>Description.</p></td>
          <td>
              <em>required</em>
          </td>
        </tr>
        <tr>
          <td><c